# 検索データ作成

[Natsume](https://hinoki-project.org/natsume/)は，名詞ー格助詞ー動詞などの構文パターンを検索したり，そのジャンル間の使用を比較したりすることができるシステムです。
ここでは，その検索機能の位置，名詞ー格助詞ー動詞の構文パターンを抽出することにします。

## 準備

すでに`requirements.txt`を`pip`などでインストール済みだったら，以下の宣言は不要です。Google Colabなどの場合は実行が必要になります（`#`をとってください）。

In [1]:
!#pip install -r requirements.txt

source: Error encountered while sourcing file “/opt/miniconda3/etc/fish/conf.d/conda.fish”:
source: No such file or directory


## Ginza

GinzaはSpaCyを裏で使っているので，SpaCyと使用がほとんど変わりません。ただし，一部追加機能があります。

In [1]:
import spacy

# spacy.prefer_gpu() # GPU搭載ならここでコメントを外す

nlp = spacy.load('ja_ginza') # ja_ginzaかja_ginza_electra
doc = nlp('銀座でランチを食べよう。')
for sent in doc.sents:
    for token in sent:
        print(token.i, token.orth_, token.lemma_, token.pos_,
              token.tag_, token.dep_, token.head.i)

0 銀座 銀座 PROPN 名詞-固有名詞-地名-一般 obl 4
1 で で ADP 助詞-格助詞 case 0
2 ランチ ランチ NOUN 名詞-普通名詞-一般 obj 4
3 を を ADP 助詞-格助詞 case 2
4 食べよう 食べる VERB 動詞-一般 ROOT 4
5 。 。 PUNCT 補助記号-句点 punct 4


## 係り受けの例

今回対象としている名詞ー格助詞ー動詞（NPV）パターンは係り受け構造の中で，どのように現れるか，簡単な例で示せます。
SpaCy/Ginzaで使用される係り受け構造の定義はUniversal Dependencies 2をご参照ください。

In [2]:
from spacy import displacy
displacy.render(doc)

## 係り受け関係の抽出

係り受け関係はSpaCyでDependencyMatcherという機能で検索できます。

-   <https://spacy.io/usage/rule-based-matching#dependencymatcher>

Semgrexの記号を使うことによって，係り受け構造の定義がわりと自由にできます。

```
SYMBOL	DESCRIPTION
A < B	A is the immediate dependent of B.
A > B	A is the immediate head of B.
A << B	A is the dependent in a chain to B following dep → head paths.
A >> B	A is the head in a chain to B following head → dep paths.
A . B	A immediately precedes B, i.e. A.i == B.i - 1, and both are within the same dependency tree.
A .* B	A precedes B, i.e. A.i < B.i, and both are within the same dependency tree (not in Semgrex).
A ; B	A immediately follows B, i.e. A.i == B.i + 1, and both are within the same dependency tree (not in Semgrex).
A ;* B	A follows B, i.e. A.i > B.i, and both are within the same dependency tree (not in Semgrex).
A $+ B	B is a right immediate sibling of A, i.e. A and B have the same parent and A.i == B.i - 1.
A $- B	B is a left immediate sibling of A, i.e. A and B have the same parent and A.i == B.i + 1.
A $++ B	B is a right sibling of A, i.e. A and B have the same parent and A.i < B.i.
A $-- B	B is a left sibling of A, i.e. A and B have the same parent and A.i > B.i.
```

In [3]:
from spacy.matcher import DependencyMatcher

In [4]:
pattern = [
    # anchor token: VERB
    {
        "RIGHT_ID": "verb",
        "RIGHT_ATTRS": {"POS": "VERB"}
    },
    #
    {
        "LEFT_ID": "verb",
        "REL_OP": ">",
        "RIGHT_ID": "noun",
        "RIGHT_ATTRS": {"DEP": {"IN": ["obl", "obj"]}}
    },
    #
    {
        "LEFT_ID": "noun",
        "REL_OP": ">",
        "RIGHT_ID": "case_particle",
        "RIGHT_ATTRS": {"DEP": "case",
                        "LEMMA": {"IN": ["が", "を", "に", "で", "から", "より", "と", "へ"]}}
    }
]

matcher = DependencyMatcher(nlp.vocab)
matcher.add("NPV", [pattern])
matches = matcher(doc)
matches


[(3339677041152928740, [4, 0, 1]), (3339677041152928740, [4, 2, 3])]

In [5]:
for match in matches:
    match_id, token_ids = match
    for i in range(len(token_ids)):
        print(pattern[i]["RIGHT_ID"] + ":", doc[token_ids[i]].text, doc[token_ids[i]].lemma_)
    print()


verb: 食べよう 食べる
noun: 銀座 銀座
case_particle: で で

verb: 食べよう 食べる
noun: ランチ ランチ
case_particle: を を



In [6]:
def extract_npv(doc):
   matches = matcher(doc)
   npvs = []
   for _, token_ids in matches:
        npvs.append((doc[token_ids[1]].lemma_, doc[token_ids[2]].lemma_, doc[token_ids[0]].lemma_))
   return npvs

In [7]:
extract_npv(doc)

[('銀座', 'で', '食べる'), ('ランチ', 'を', '食べる')]

## TEDトークコーパスの作成

Hugginfaceのdatasetsを使って，TEDトークの日本語に翻訳された字幕をコーパス化します。
データセットのページは以下：

-   <https://huggingface.co/datasets/ted_talks_iwslt>

In [8]:
from datasets import load_dataset
ted_dataset_2014 = load_dataset("ted_talks_iwslt", language_pair=("en", "ja"), year="2014")
ted_dataset_2015 = load_dataset("ted_talks_iwslt", language_pair=("en", "ja"), year="2015")
ted_dataset_2016 = load_dataset("ted_talks_iwslt", language_pair=("en", "ja"), year="2016")

Using custom data configuration en_ja_2014-29fbff389ea2dff6
Reusing dataset ted_talks_iwslt (/home/bor/.cache/huggingface/datasets/ted_talks_iwslt/en_ja_2014-29fbff389ea2dff6/1.1.0/43935b3fe470c753a023642e1f54b068c590847f9928bd3f2ec99f15702ad6a6)
100%|██████████| 1/1 [00:00<00:00, 217.05it/s]
Using custom data configuration en_ja_2015-53b0fd110d70daa0
Reusing dataset ted_talks_iwslt (/home/bor/.cache/huggingface/datasets/ted_talks_iwslt/en_ja_2015-53b0fd110d70daa0/1.1.0/43935b3fe470c753a023642e1f54b068c590847f9928bd3f2ec99f15702ad6a6)
100%|██████████| 1/1 [00:00<00:00, 613.74it/s]
Using custom data configuration en_ja_2016-2ba3e9afcbc60ece
Reusing dataset ted_talks_iwslt (/home/bor/.cache/huggingface/datasets/ted_talks_iwslt/en_ja_2016-2ba3e9afcbc60ece/1.1.0/43935b3fe470c753a023642e1f54b068c590847f9928bd3f2ec99f15702ad6a6)
100%|██████████| 1/1 [00:00<00:00, 579.24it/s]


In [9]:
ted_corpus = [d['ja'] for d in ted_dataset_2014['train']['translation']] + [d['ja'] for d in ted_dataset_2015['train']['translation']] + [d['ja'] for d in ted_dataset_2016['train']['translation']]
len(ted_corpus)

10114

In [10]:
ted_corpus

['野生生物の保護に尽力するボイド・ヴァーティは「自然の大聖堂は人間性の最高の部分を映し出してくれる鏡である」と話します。ヴァーティが野生動物と人間の繋がり、そして「あなたがいるから私がいる」という意味の“ubuntu”について語ります。彼は自身の講演を南アフリカのリーダーで寛容の精神を体現した故ネルソン・マンデラ元大統領に捧げます。',
 'ボイド・ヴァーティ: ネルソン・マンデラが教えてくれたこと',
 'デイビッド・ラングは、独学でアマチュア海洋生物学者になりました―というより、彼の代わりにロボットを海洋生物学者にしました。素敵なトークで、TEDフェローのラングは、どうやって海洋愛好家たちが力を合わせて、オープンソースで低コストの水中探査機を作り上げたか語ります。',
 'デイビッド・ラング: 私の水中探査ロボット',
 '興味深い統計があります。イギリスでは、男性短期受刑者の63％は、出所後１年以内に再犯し、刑務所に逆戻りしてしまうというのです。社会復帰を支援するためには、職業訓練や教育、心理カウンセリングなどが必要で、それができれば大きな成果を上げるでしょう。しかし、政府にはそんな財源がありません。トビー・エクルズは、この状況を覆すための画期的なアイデア「ソーシャル・インパクト・ボンド（社会インパクト債権）」について話します。この債権は今までにないもので、民間のお金を使って社会問題の解決に取り組み、さらに、それが成功すれば、政府が利子を付けて投資家に資金を払い戻すというものです。',
 'トビー・エクルズ: 投資で社会変革を',
 '人の脳は奇妙なものです。体のサイズの割には、不思議に大きく、その重さの割には、使うエネルギーはとてつもないものです。そして、その大脳皮質は、おそろしく密度の高いものです。何故でしょう？神経科学者のスザーナ・エルクラーノ＝アウゼル女史は、その神秘の世界に私たちを誘い、「脳スープ」を作ることで、人の脳の謎を紐解いて行き、驚く結果に辿り着きます。',
 'スザーナ・エルクラーノ＝アウゼル: 人の脳は、何がそんなに特別なのでしょうか？',
 '「人生の出来事が次々と目の前で起きています。私たちは、形も定まらない体験の流れを捉えて、そこから何らかの意味を引き出さなければなりません」 教育心理学者のピーター・ドゥーリトルは、この愉快でため

In [11]:
with open('ted_corpus.txt', 'w') as f:
    f.write('\n'.join(ted_corpus))

In [12]:
from itertools import chain
ted_npvs = list(chain.from_iterable(extract_npv(doc) for doc in nlp.pipe(ted_corpus)))
# GPU 1m38s

In [13]:
len(ted_npvs)

34910

In [14]:
# 格助詞ごとの項目数を調べるなら
from collections import Counter

Counter(npv[1] for npv in ted_npvs)

Counter({'に': 10076,
         'を': 17402,
         'で': 4142,
         'から': 1118,
         'と': 1649,
         'へ': 238,
         'が': 198,
         'より': 87})

In [15]:
import pandas as pd

## NPVデータの保存

検索インターフェースでは今回NPVパターンのみを検索するため，そのデータのみをCSV形式に書き出す。

In [16]:
df = pd.DataFrame.from_records(ted_npvs, columns=["n", "p", "v"])
df

,n,p,v
0,保護,に,尽力
1,部分,を,映し出す
2,ubuntu,に,語る
3,精神,を,体現
4,講演,を,捧げる
...,...,...,...
34905,正義,を,求める
34906,闘い,に,説明
34907,地区,に,受ける
34908,被害,を,受ける


In [17]:
df.to_csv("ted_npvs.csv", index=False)